In [1]:
import os

os.chdir("../")
%pwd

'd:\\DHIRAJ\\Data_Science\\Jupyter_Workspace\\Projects\\SpaceX-Falcon-9-first-stage-Landing-Prediction'

In [2]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path
    data_path: Path
    evaluation_result_path: Path

In [3]:
from SpaceXF9LandingPred.constants import *
from SpaceXF9LandingPred.utils.common import read_yaml, create_directories

In [ ]:
class ConfigurationManager:
    def __init__(self, 
                config_filepath=CONFIG_FILE_PATH, 
                params_filepath=PARAMS_FILE_PATH):
        
        self.config_file=read_yaml(config_filepath)
        self.params_file=read_yaml(params_filepath)

        create_directories([self.config_file.artifacts_root])
    
    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        
        config=self.config_file.model_evaluation
        params=self.params_file.evaluation_params
        create_directories([config.root_dir])

        model_evaluation_config=ModelEvaluationConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
            evaluation_result_path=config.evaluation_result_path
        )

        return model_evaluation_config, params
    

In [ ]:
import pandas as pd
import numpy as np

from SpaceXF9LandingPred.logging import logger

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier


class ModelEvaluation:
    def __init__(self, config:ModelEvaluationConfig, params):
        self.config=config
        self.params=params
        self.scaler=StandardScaler()

    def evaluate_models(self):

        data = pd.read_csv(self.config.data_path)
        X = data.drop(columns=['Class'], axis=1)
        Y = data['Class'].to_numpy()

        X = self.scaler.fit_transform(X)

        X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=2)

        results={"Models":[],
                 "Accuracy Score":[]}

        models=[LogisticRegression(), SVC(), DecisionTreeClassifier(), KNeighborsClassifier()]
        
        for model in models:
            mdl=model
            model_name=mdl.__class__.__name__
            parameters=dict(self.params[model_name.lower()])
            cv=GridSearchCV(mdl, parameters, cv=10)
            cv.fit(X_train, Y_train)
            accuracy=cv.score(X_test, Y_test)
            results["Models"].append(model_name)
            results["Accuracy Score"].append(accuracy)

        result_df=pd.DataFrame(results)

        result_df.to_csv(self.config.evaluation_result_path, index=False)

        logger.info("Model Evaluation results are saved in artifacts")

In [12]:
try:
    config = ConfigurationManager()
    model_evaluation_config, params = config.get_model_evaluation_config()
    model_evaluation = ModelEvaluation(config=model_evaluation_config, params=params)
    model_evaluation.evaluate_models()
except Exception as e:
    raise e

[2025-08-11 15:37:45,690: INFO: common: yaml file : config\config.yaml loaded successfully]
[2025-08-11 15:37:45,730: INFO: common: yaml file : params.yaml loaded successfully]
[2025-08-11 15:37:45,737: INFO: common: created directory at : artifacts]
[2025-08-11 15:37:45,740: INFO: common: created directory at : artifacts/model_evaluation]
[2025-08-11 15:37:52,625: INFO: 1859856227: Processed Data file has been created in artifacts]
